In [1]:
# Keep notebook output clean for now
import warnings
warnings.filterwarnings('ignore')

In [2]:
from langchain_openai import ChatOpenAI
from langchain.chains import VectorDBQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
# from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

from pathlib import Path

In [3]:
loader = TextLoader(Path("TX_SB2102.txt").absolute(), autodetect_encoding=True)
data = loader.load()
data[0].metadata = {'keywords': 'TX_SB2102'}
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
splits = text_splitter.split_documents(data)
# embeddings = GPT4AllEmbeddings()
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectordb = Chroma.from_documents(splits, embeddings)

In [5]:
llm = ChatOpenAI(base_url="http://localhost:1234/v1", api_key="not-needed", temperature=0.7, max_tokens=1000, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])
qa = VectorDBQA.from_chain_type(llm=llm, chain_type="stuff", vectorstore=vectordb, k=10, return_source_documents=True)

In [6]:
#Use semicolon to suppress additional printed output
def askLLM(query):
    result = qa({"query": query});
    print("\n --- SOURCES --- \n")
    for idx, doc in enumerate((result['source_documents'])):
        print(f"Source {idx+1}:\n", doc.page_content, "\n")

In [7]:
askLLM("What was in TX SB 2102?")

/Users/phil/.local/share/virtualenvs/LLM_RAG-sfbpCevB/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


TX SB 2102 was an act relating to the initial registration and inspection period for certain rental vehicles. It included provisions for a three-year emissions inspection period for certain vehicles, authorization for fees, and submission of revisions to the Texas Commission on Environmental Quality for approval by the United States Environmental Protection Agency for commercial fleet buyers. The bill also required that fees collected were to provide the same revenue as if the vehicle was inspected annually or biennially, and added a new section to Chapter 502, Transportation Code, regarding vehicle inspection and maintenance programs approved by the United States Environmental Protection Agency under Section 548.301 and the Clean Air Act.
 --- SOURCES --- 

Source 1:
 S.B. No. 2102
 
 
 
 	
AN ACT
 	relating to the initial registration and inspection period for
 	certain rental vehicles; authorizing fees.
 	       BE IT ENACTED BY THE LEGISLATURE OF THE STATE OF TEXAS:
 	       SECTIO

In [8]:
askLLM("When was TX SB 2102 passed?")

TX SB 2102 was passed on May 17, 2023.
 --- SOURCES --- 

Source 1:
 S.B. No. 2102
 
 
 
 	
AN ACT
 	relating to the initial registration and inspection period for
 	certain rental vehicles; authorizing fees.
 	       BE IT ENACTED BY THE LEGISLATURE OF THE STATE OF TEXAS:
 	       SECTION 1.  Section 382.0622(a), Health and Safety Code, is 

Source 2:
 provides for a three-year emissions inspection period for a vehicle
 	described by Section 548.1025(a), Transportation Code:
 	             (1)  the system implemented by the Department of Public
 	Safety of the State of Texas under Subsection (d) shall provide for 

Source 3:
 amended by amending Subsection (d) and adding Subsection (d-2) to
 	read as follows:
 	       (d)  Except as provided by Subsection (d-2), on [On] adoption
 	of a resolution by the commission and after proper notice, the
 	Department of Public Safety of the State of Texas shall implement a 

Source 4:
 the state under Section 548.509.
 	       SECTION 7.  (a)  As

In [9]:
askLLM("How much will this bill cost citizens?")

The bill amends several sections of the Health and Safety Code and Transportation Code related to vehicle inspections and fees. The fees for motor vehicle inspections are set at $12.50 for non-mopeds and $5.75 for mopeds, as specified in Section 6 of the bill. However, Section 2 amends Sections 382.202 and 548.5035 of the Transportation Code, which will result in an additional $2 fee per registration period for vehicle inspections. Additionally, Section 185 of the federal Clean Air Act requires certain fees to be collected, and these fees are not specified in the text provided. Therefore, it is unclear what the total cost to citizens would be as a result of this bill without additional information about the Clean Air Act fees.
 --- SOURCES --- 

Source 1:
 amended to read as follows:
 	       (a)  Clean Air Act fees consist of:
 	             (1)  fees collected by the commission under Sections
 	382.062, 382.0621, 382.202, and 382.302 and as otherwise provided
 	by law;
 	            